In [1]:
import pandas as pd
import numpy as np
import argparse
import datetime
#import GPUtil
import random
import keras
import glob
import time
import sys
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, MaxPooling2D, Flatten
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications.xception import Xception
from keras.initializers import Orthogonal
from keras.utils import to_categorical
from keras.preprocessing import image
from generators import DataGenerator
from scipy.misc import imresize
from keras.models import Model
from keras import backend as K
from keras import optimizers
from skimage import io
from keras.callbacks import CSVLogger, Callback, EarlyStopping
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,ReduceLROnPlateau
from keras.callbacks import TensorBoard, ModelCheckpoint

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
base=load_model('Xception_model(71).h5')

C:\Users\Biototem_1\Anaconda3\envs\huajia\lib\site-packages\keras\engine\saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
top_model = Sequential()
top_model.add(base)
top_model.add(MaxPooling2D(pool_size=(2,2)))
top_model.add(Flatten())
top_model.add(Dropout(0.5))
top_model.add(Dense(128, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax',kernel_initializer=Orthogonal()))
top_model.compile(optimizer = "SGD", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [5]:
top_model.load_weights('./Xception/Xception_decay.hdf5')

In [6]:
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total para

In [27]:
model=Model(inputs=top_model.input,outputs=top_model.get_layer('max_pooling2d_1').output)
#不知道为什么，无法把模型的输出定义到'Xception'层那里

In [28]:
train_data_dir = './raw/train'
validation_data_dir = './raw/validation'
test_data_dir = './raw/test'
test_datagen= DataGenerator(rescale=1./255)
batch_size = 20
img_width, img_height = 71,71
def extract_features(directory,sample_count):

    features = np.zeros(shape=(sample_count,1,1,2048))
    labels = np.zeros(shape=(sample_count))
    generator = test_datagen.flow_from_directory(directory,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='binary')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features[i*batch_size: (i+1)*batch_size] = features_batch
        labels[i*batch_size: (i+1)*batch_size] = labels_batch
        i+=1
        if i*batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels=extract_features(train_data_dir,5289 )
validation_features, validation_labels=extract_features(validation_data_dir,900)
test_features, test_labels=extract_features(test_data_dir,900)

Found 5289 images belonging to 3 classes.
Found 900 images belonging to 3 classes.
Found 900 images belonging to 3 classes.


In [29]:
train_features_1 = np.reshape(train_features, (5289  , 1 * 1 * 2048))
validation_features_1 = np.reshape(validation_features, (900 , 1 * 1 * 2048))
test_features_1 = np.reshape(test_features, (900, 1 * 1 * 2048))

In [30]:
train_features_2 = pd.DataFrame(train_features_1)
train_labels_2 = pd.DataFrame(train_labels)
validation_features_2 = pd.DataFrame(validation_features_1)
validation_labels_2 = pd.DataFrame(validation_labels)
test_features_2 = pd.DataFrame(test_features_1)
test_labels_2 = pd.DataFrame(test_labels)

In [31]:
train_features_2.to_csv("./feature_extract_70/train_features.csv",index=False,sep=',')
train_labels_2.to_csv("./feature_extract_70/train_labels.csv",index=False,sep=',')
validation_features_2.to_csv("./feature_extract_70/validation_features.csv",index=False,sep=',')
validation_labels_2.to_csv("./feature_extract_70/validation_labels.csv",index=False,sep=',')
test_features_2.to_csv("./feature_extract_70/test_features.csv",index=False,sep=',')
test_labels_2.to_csv("./feature_extract_70/test_labels.csv",index=False,sep=',')

In [14]:
validation_features_2.shape

(5289, 2048)

In [10]:
dataframe = pd.DataFrame(test_labels)

In [11]:
dataframe.shape

(900, 1)

In [22]:
df=dataframe.values

In [12]:
dataframe.to_csv("test_label.csv",index=False,sep=',')

In [30]:
import pandas as pd
data = pd.read_csv('test.csv')

In [31]:
data

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.397491,0.000000,0.644172,0.000000,0.100749,0.000000,0.000000,0.440752,0.165393,0.000000,...,0.000000,0.009638,0.000000,0.140210,0.000000,0.469266,0.116161,0.000000,0.299930,0.000000
1,0.508380,0.000000,0.737333,0.236262,0.000000,0.089112,0.000000,0.103054,0.000000,0.653596,...,0.020008,0.208333,0.000000,0.000000,0.000000,0.773704,0.000000,0.000000,0.354649,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.190611,0.248353,0.000000,0.000000,0.000000,0.393802,0.000000,...,0.109565,0.068896,0.000000,0.000000,0.080086,0.000000,0.063701,0.097070,0.000000,0.000000
4,0.302596,0.000000,0.889637,0.114802,0.328262,0.011312,0.025108,0.072357,0.276672,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.488742,0.114848,0.192800,0.323198,0.112226
5,0.000000,0.000000,0.000000,0.000156,0.033415,0.014480,0.016330,0.000000,0.066953,0.000000,...,0.000000,0.000000,0.123405,0.000000,0.000000,0.000000,0.044508,0.002084,0.000000,0.000000
6,0.000000,0.557206,0.000000,0.423665,0.461033,0.044400,0.000000,0.779228,0.908393,0.103237,...,0.000000,1.028065,0.000000,0.000000,0.000000,0.000000,0.330319,0.478338,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.711433,0.195647,0.916049,0.035552,0.427676,0.276987,0.000000,0.782506,0.503785,0.452543,...,0.000000,0.497537,0.222603,0.350393,0.000000,0.458052,0.335408,0.346286,0.676595,0.260884
9,0.306172,0.000000,0.508875,0.167899,0.211366,0.373532,0.000000,0.000000,0.109371,0.000000,...,0.000000,0.000000,0.159044,0.033851,0.000000,0.546058,0.085949,0.165824,0.102962,0.160842


In [16]:
from sklearn.svm import SVC
from sklearn import cross_validation,metrics

C:\Users\Biototem_1\Anaconda3\envs\huajia\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [90]:
svm_model = SVC(C=3,gamma=0.1,kernel='poly',class_weight='balanced')

In [129]:
svm_model = SVC(C=3,gamma=0.008,kernel='poly',class_weight='balanced',coef0=0.008,cache_size=500)
svm_model.fit(train_features,train_labels)
print(svm_model)
y_pred=svm_model.predict(test_features)
print('\n'+'the f1_score is : '+ str(metrics.f1_score(test_labels,y_pred,average='macro')))

SVC(C=3, cache_size=500, class_weight='balanced', coef0=0.008,
  decision_function_shape='ovr', degree=3, gamma=0.008, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

the f1_score is : 0.8363730217544186


In [121]:
svm_model = SVC(C=3,gamma=0.008,kernel='poly',class_weight='balanced',coef0=0.008)
svm_model.fit(train_features,train_labels)
print(svm_model)
y_pred=svm_model.predict(test_features)
print('\n'+'the f1_score is : '+ str(metrics.f1_score(test_labels,y_pred,average='macro')))

SVC(C=3, cache_size=200, class_weight='balanced', coef0=0.008,
  decision_function_shape='ovr', degree=3, gamma=0.008, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

the f1_score is : 0.8363730217544186


In [ ]:
svm_model = SVC(C=3,gamma=0.008,kernel='poly',class_weight='balanced',coef0=0.008)
svm_model.fit(train_features,train_labels)
print(svm_model)
y_pred=svm_model.predict(test_features)
print('\n'+'the f1_score is : '+ str(metrics.f1_score(test_labels,y_pred,average='macro')))

In [25]:
svm_model = SVC(C=3,gamma=0.008,kernel='poly',class_weight='balanced',coef0=0.008)
svm_model.fit(train_features,train_labels)
print(svm_model)
y_pred=svm_model.predict(test_features)
print('\n'+'the f1_score is : '+ str(metrics.f1_score(test_labels,y_pred,average='macro')))

SVC(C=3, cache_size=200, class_weight='balanced', coef0=0.008,
  decision_function_shape='ovr', degree=3, gamma=0.008, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

the f1_score is : 0.8393451977162254


In [19]:
#y_pred=svm_model.predict(test_features)
print(metrics.accuracy_score(test_labels,y_pred))

0.8466666666666667


In [ ]:
y_pred=svm_model.predict(train_features)
print(metrics.accuracy_score(test_labels,y_pred))

In [54]:
print(metrics.f1_score(test_labels,y_pred,average='weighted'))

0.3363962945905298
